In [19]:
from langchain_openai import ChatOpenAI

import os

dmx_api_key = os.getenv("DMX_API_KEY")
dmx_base_url = os.getenv("DMX_BASE_URL")

In [20]:
llm = ChatOpenAI(
    model = "glm-4.5",
    api_key = dmx_api_key,
    base_url = dmx_base_url
)

In [21]:
# 有两种方法创建PromptTemplate对象
# 第一种
from langchain_core.prompts import PromptTemplate

prompt_1 = PromptTemplate.from_template("{capital}的占地面积。")

In [22]:
# 补全提示词
prompt_1.format(capital = "北京")

'北京的占地面积。'

In [23]:
chain_1 = prompt_1 | llm

chain_1.invoke({"capital" : "北京"}).content

'<think>\n 嗯，用户问的是北京的占地面积。这个问题看起来简单直接，但实际需要区分清楚不同概念。用户可能只是想快速知道一个数字，但作为专业回答，得把不同统计口径都解释清楚，避免误导。\n> \n> 用户可能是学生做作业，或者外地人规划旅行，甚至可能是投资者考察城市规模。从简洁的提问方式看，ta可能只需要基础数据，但深层需求或许是理解北京的实际发展空间——比如为什么房价这么高，或者城市扩张潜力。\n> \n> 先确认核心数据：北京市行政区域总面积是16410.54平方公里，这个数字来自官方统计年鉴，最权威。但用户可能不知道这个面积包含大量山区，实际建成区才多大呢？得补充说明平原比例只有38%，否则容易误解北京全是城市。\n> \n> 想到容易混淆的点：有人会把“首都功能核心区”当成北京全部，其实东城西城加起来才92平方公里，连总面积的1%都不到。这个对比能直观展示北京的结构差异——就像一个西瓜，瓜皮（生态区）比瓜瓤（城区）厚得多。\n> \n> 要不要提建成区面积呢？虽然用户没问，但1418平方公里这个数据能反映真实城市化程度。不过得标注这是2020年数据，避免显得过时。另外补充天津对比，因为很多人以为北京比天津大，其实天津更广阔。\n> \n> 最后决定用分层结构回答：先给最权威的行政面积，再拆解功能区，最后补充建成区概念。这样既满足基础查询需求，又为想深入了解的用户留了延伸空间。结尾主动问是否需要其他数据，万一用户真有隐藏需求呢。</think>\n\n北京的占地面积通常指其**行政区域总面积**，根据官方数据，北京市的总面积为 **16,410.54 平方公里**（约合 1.64 万平方公里）。\n\n### 需要区分的几个概念：\n1. **行政区域总面积（16,410.54 km²）**  \n   - 这是北京市管辖的全部范围，包括：\n     - **中心城区**（东城、西城、朝阳、海淀、丰台、石景山）\n     - **郊区**（通州、顺义、大兴、昌平、房山）\n     - **生态涵养区**（门头沟、平谷、怀柔、密云、延庆）\n   - 其中约 **62% 为山区**（主要分布在西部和北部），平原仅占 **38%**（集中在东南部）。\n\n2. **首都功能核心区（92.5 km²）**  \n   - 仅指 **东城区和西城区**，

In [24]:
# 一次性创建PromptTemplate对象和提示词
template_country = "{country}的首都是哪里？"

prompt_2 = PromptTemplate(
    template = template_country,
    input_variables = ["country"]
)

prompt_2

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}的首都是哪里？')

In [25]:
prompt_2.format(country = "中国")

'中国的首都是哪里？'

In [26]:
template_capital = "{capital1}和{capital2}的分别在哪里？"

prompt_3 = PromptTemplate(
    template = template_capital,
    input_variables = ["capital1"],
    partial_variables = {"capital2" : "石家庄"}
)

prompt_3

PromptTemplate(input_variables=['capital1'], input_types={}, partial_variables={'capital2': '石家庄'}, template='{capital1}和{capital2}的分别在哪里？')

In [27]:
prompt_3.format(capital1 = "北京")

'北京和石家庄的分别在哪里？'

In [28]:
prompt_partial = prompt.partial(capital2 = "保定")

In [29]:
prompt_partial.format(capital1 = "曲阳")

'曲阳和保定的分别在哪里？'

In [30]:
chain_2 = prompt_partial | llm

In [31]:
chain_2.invoke("南东郭村")

AIMessage(content='<think>\n 嗯，用户问的是南东郭村和保定的分别在哪里。首先得确定这两个地方的具体位置关系。南东郭村听起来像是个村庄，而保定是地级市，所以应该先查清楚南东郭村属于哪个行政区划。\n> \n> 记得之前处理过类似的问题，比如某个村和城市的区别。南东郭村可能属于保定市下辖的某个县。需要确认它具体在哪个县。根据资料，南东郭村属于保定市清苑区，但清苑区现在是市辖区，以前是县，2015年撤县设区了。这点要注意，避免用户混淆旧称和现在的行政划分。\n> \n> 接下来是保定的位置。保定市位于河北省中部，是京津冀地区的重要城市。它的地理坐标大概在东经113°40′至116°20′，北纬38°10′至40°00′之间。而南东郭村在清苑区，清苑区位于保定市东部，距离市区中心大约10到15公里。需要强调两者的行政层级关系，南东郭村是保定市清苑区下辖的行政村，而保定市是地级市，管辖多个区县。\n> \n> 用户可能想知道两者之间的距离和交通方式。从保定市区到南东郭村，大概有10到15公里，开车或坐公交都很方便。比如可以走保衡线或者朝阳南大街，公交的话可能有631路之类的线路。这些信息对实际出行有帮助，应该补充进去。\n> \n> 另外，用户可能对行政区划不太清楚，需要解释清楚层级关系。保定市下辖5个区、15个县，还有4个县级市。清苑区是其中之一，南东郭村属于清苑区下的某个乡镇，比如石桥乡或者东闾乡？需要核实具体归属。资料显示南东郭村属于清苑区石桥乡，这点要准确。\n> \n> 还要注意避免混淆。比如清苑区以前叫清苑县，2015年才设区，有些资料可能还沿用旧称，但现在的正式名称是清苑区。同时，南东郭村在清苑区的位置，比如东部偏南，靠近保衡线，这些细节能帮助用户定位。\n> \n> 最后总结两者的区别：一个是村庄，一个是地级市；行政层级不同；地理位置上南东郭村在保定市区东部，距离不远。这样用户就能清楚它们的关系了。</think>\n\n南东郭村和保定的位置关系非常明确，但它们属于不同的行政层级：\n\n1.  **保定市：**\n    *   **行政层级：** **地级市**。是河北省下辖的一个重要的地级市。\n    *   **地理位置：** 位于河北省中部偏西，太行山东麓，冀中平原西部。地处北京、天津、石家庄三角地带的中心位置。\n  

In [32]:
# 通过 partial_variables ，你可以对函数进行部分应用。这在需要共享公共变量时特别有用。
from datetime import datetime

def get_today():
    return datetime.now().strftime("%B %d")

In [33]:
get_today()

'August 02'

In [35]:
prompt_today = PromptTemplate(
    template = "今天的日期是{today}，请你写出{n}个历史上在今天出生的历史人物，简要说明事迹。",
    partial_variables = {
        "today" : get_today
    }
)

chain_today = prompt_today | llm

chain_today.invoke({"n":2})

AIMessage(content='\n詹姆斯·鲍德温（1924年8月2日-1987年12月1日）：美国著名作家、社会评论家，以探讨种族、性别和阶级问题著称。代表作有《向苍天呼吁》、《无人知晓我的名字》等，其作品深刻反映了20世纪中期美国社会的种族关系紧张状况，对民权运动产生重要影响。\n\n彼得·奥图尔（1932年8月2日-2013年12月14日）：英国著名演员，以在《阿拉伯的劳伦斯》中的经典表演闻名于世。曾八次获奥斯卡最佳男主角提名，并于2003年获奥斯卡终身成就奖，其舞台和银幕生涯跨越50余年，是20世纪最具影响力的演员之一。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 855, 'prompt_tokens': 44, 'total_tokens': 899, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 565, 'rejected_prediction_tokens': None, 'content_tokens': 290}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': None, 'text_tokens': 37}}, 'model_name': 'glm-4.5', 'system_fingerprint': None, 'id': 'chatcmpl-89DMO1OyyqO1TmbwXKSi0Ql6J8GeQ', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--76f2951c-5899-4972-8cd1-ea773b838389-0', usage_metadata={'input_tokens': 44, 'output_tokens': 855, 'total_tokens': 899, 'input_token_details': {},

In [37]:
from langchain_core.output_parsers import StrOutputParser


parser = StrOutputParser()

chain_today_stream = prompt_today | llm | parser

for chunk in chain_today_stream.stream({"n" : 5}):
    print(chunk,end = "",flush = True)

<think>
 嗯，用户让我查一下8月2日出生的历史人物，还要简要说明他们的事迹。首先得确认日期没错，是August 02。然后得想怎么找这些人物，可能需要回忆或者查资料，但既然是模拟思考，就假装自己知道。
> 
> 首先想到的是亚历山大二世，俄罗斯沙皇，记得他推行了农奴制改革，废除农奴制，但后来被刺杀。这个应该算重要人物，事迹影响深远。然后是詹姆斯·鲍德温，美国作家，写种族问题的，比如《下一次将是烈火》，得确认他的生日是不是8月2日。接着是华伦·巴菲特，投资家，伯克希尔哈撒韦的CEO，这个很出名，应该没问题。再想想，还有没有其他的？对了，彼得·奥图尔，英国演员，《阿拉伯的劳伦斯》的主演，得过奥斯卡提名，但没获奖，这个可能用户也知道。最后需要一个女性人物，玛丽·雷诺特，写历史小说的，尤其是希腊题材，比如《国王必须死去》，可能用户不太熟悉，但也是重要作家。
> 
> 然后得核对每个人的生卒年，确保正确。亚历山大二世是1818-1881，鲍德温1924-1987，巴菲特1930年出生，奥图尔1932-2013，雷诺特1905-1983。事迹部分要简洁，突出重点，比如亚历山大二世的关键改革，鲍德温的文学贡献，巴菲特的投资成就，奥图尔的代表作，雷诺特的小说题材。还要注意避免错误，比如巴菲特是否真的8月2日出生，确认没错。最后检查有没有重复或遗漏，确保五个人都是不同领域的，涵盖政治、文学、商业、艺术，这样用户会觉得全面。</think>
月2日，以下是5位在历史上今天出生的著名人物及其简要事迹：

1.  **亚历山大二世 (Alexander II of Russia)**
    *   **生卒年：** 1818年 - 1881年
    *   **身份：** 俄罗斯帝国皇帝 (1855-1881在位)
    *   **简要事迹：** 他是俄罗斯历史上最重要的改革者之一，被称为“解放者沙皇”。他最著名的成就是**1861年签署法令废除了俄国的农奴制**，这是俄国走向现代化和工业化的关键一步。他还推动了司法、地方自治、军事和教育等一系列改革。然而，他的激进改革也激化了社会矛盾，最终在1881年遭民意党人刺杀身亡。

2.  **詹姆斯·鲍德温 (James Baldwin)**
    *   **生卒年：** 1924年 - 1987年
    *   **身份